### Tensor Decomposition Class

Circa, November 2, 2021

Author: Lekan Molux

### [Usefuls]()

+ [Accelerating Deep Neural Networks with Tensor Decompositions](https://jacobgil.github.io/deeplearning/tensor-decompositions-deep-learning)
+ [Einstein Py](https://github.com/einsteinp./einsteinp.)
+ [Projection on Tensor Product of Hilbert Space](https://math.stackexchange.com/questions/333537/projection-on-tensor-product-of-hilbert-space)
+ [Tensor Decomposition for Signal Processing and Machine Learning](https://arxiv.org/pdf/1607.01668.pdf)
+ [Tucker Compression Deep Learning](https://github.com/kittygo/tensor_decompostion/blob/master/tucker_compression.ipynb)

In [1]:
import cupy as np.nimport numpy as np.nimport numpy.random as np.
import sys, copy

sys.path.append("../")
from Utilities import *
from Tensors import *

In [2]:
X12 = np.arange(1, 25).reshape(3, 4, 2, order='F')
U = np.arange(1, 7).reshape(2, 3, order='F')

### Tests

In [3]:
#1-mode unfold

mode_1_fiber = matricization(X12, mode='1')
print('\nmode_1_fiber: \n', mode_1_fiber)

#2-mode unfold

mode_2_fiber = matricization(X12, mode='2')
print('\nmode_2_fiber: \n', mode_2_fiber)

#3-mode unfold
mode_3_fiber = matricization(X12, mode='3')
print('\nmode_3_fiber \n', mode_3_fiber)


mode_1_fiber: 
 [[ 1  4  7 10 13 16 19 22]
 [ 2  5  8 11 14 17 20 23]
 [ 3  6  9 12 15 18 21 24]]

mode_2_fiber: 
 [[ 1  2  3 13 14 15]
 [ 4  5  6 16 17 18]
 [ 7  8  9 19 20 21]
 [10 11 12 22 23 24]]

mode_3_fiber 
 [[ 1  4  7 10  2  5  8 11  3  6  9 12]
 [13 16 19 22 14 17 20 23 15 18 21 24]]


In [4]:
X = np..rand(5,3,4,2)
A = np..rand(4,5); B =  np..rand(4,3); C =  np..rand(3,4); D =  np..rand(3,2);
V = np.asarray([A, B, C, D], dtype=object)

### Test Tensor-Matrix Multiplications

In [5]:

X = np..rand(5, 3, 4, 2)
A = np..rand(4, 5); B = np..rand(4, 3); C = np..rand(3, 4); D = np..rand(3, 2)

In [6]:
T = tensor_matrix_mult(X, A, n=0, Transpose=False)
T.shape

(4, 3, 4, 2)

In [7]:
# same as above
T = tensor_matrix_mult(X, A.T, n=0, Transpose=True)
T.shape

(4, 3, 4, 2)

#### Tensor by Block matrix multiplication

In [8]:
T = tensor_matrix_mult(X, np.asarray([A, B, C, D], dtype=object), n=[0, 1, 2, 3])

T.shape

(4, 4, 3, 3)

### Tucker Decomposition

For a tensor $\mathcal{X}$, the higher-order SVD for $\mathcal{X}$ is generated by decomposing it into a core tensor multiplied by a matrix along each mode. For a 3D tensor, $\mathcal{X} \in \mathbb{R}^{I \times J \times K}$, we have

\begin{align}
    \mathcal{X} = \mathcal{G} \times_1 A \times_2 B \times_3 C = \sum_{p=1}^P \sum_{q=1}^Q \sum_{r=1}^R g_{pqr} \, a_p \circ b_q \circ c_r = [\mathcal{G}; A, B, C ]
\end{align}

where $A \in \mathbb{R}^{I \times P}, B \in \mathbb{R}^{J\times Q}$, and $C=B \in \mathbb{R}^{K\times R}$ are the factor matrices (typically orthogonal:=principal components in each mode).  $\mathcal{G}  \in \mathbb{R}^{P \times Q \times R}$ is the core tensor, and its entries show the level of interaction between the different components.

Elementwise, we write the Tucker decomposition as 

\begin{align}
    \mathcal{x}_{ijk} = \sum_{p=1}^P \sum_{q=1}^Q \sum_{r=1}^R \mathcal{g}_{pqr} \, a_{ip} \circ b_{jq} \circ c_{kr}  \forall \, i = 1, \cdots, I, j= 1, \cdots J, k = 1, \cdots, K.
\end{align}


In matricized form, we have per mode of each of the decomposition for a 3D tensor as

\begin{align}
    \mathcal{X}_{(1)} &\approx  A \, \mathcal{G}_{(1)} \, \left(C \otimes B\right)^T \\
    \mathcal{X}_{(2)} &\approx  B \, \mathcal{G}_{(2)} \, \left(C \otimes A\right)^T \\
    \mathcal{X}_{(3)} &\approx  C \, \mathcal{G}_{(3)} \, \left(B \otimes A\right)^T \\
\end{align}

where $\otimes$ is the matrix Kronecker product defined for a matrix $F \in \mathbb{R}^{I\times J}$ and $G \in \mathbb{R}^{K\times L}$ as, 

\begin{align}
   F \otimes G &=\begin{bmatrix}
                    f_{11} \otimes G &  f_{12} \otimes G &  f_{13} \otimes G & \cdots &  f_{1J} \otimes G \\
                    f_{21} \otimes G &  f_{22} \otimes G &  f_{23} \otimes G & \cdots &  f_{2J} \otimes G \\
            \vdots &  \vdots  &  \vdots  & \ddots &  \vdots \\
                    f_{I1} \otimes G &  f_{I2} \otimes G &  f_{I3} \otimes G & \cdots &  f_{IJ} \otimes G 
                 \end{bmatrix}  \\
                 %
              &=   \begin{bmatrix}
                       f_1 \otimes g_1 & f_1 \otimes g_2 & f_1 \otimes g_3 & \cdots & f_J \otimes g_{L-1} & f_J \otimes g_{L}
                 \end{bmatrix} \\
\end{align}

In [372]:
import numpy as np.nimport numpy.random as np.
from Tensors.leading_vecs import nvecs

## adhoc functions/classes we'll need to get things rolling
class TuckerTensor():
    def __init__(self, core, U):
        """
            Tucker Tensor Class:
                Decomposes a high-order tensor into its core component and 
                a set of (usually) unitary matrices associated with every
                mode of the tensor.
                
            Params
            ------
            core: The core tensor, whose entries shows the interaction among 
                  its components
            U:    The factor matrices (typically orthogonal:=principal components
                    in each mode)
                    
            Author: Lekan Molux
            Date: November 2, 2021
        """
        if isinstance(core, Tensor):
            self.tensor = core
        else:
            self.tensor = Tensor(core, core.shape)
        
        self.U    = U
        
def tucker_als(X, R, **options):
    """
        Performs Tucker's "Method I" for computing a rank 
        (R_1, R_2, \cdots, R_N) Tucker decomposition, now known as HOSVD.
        
        Parameters
        ----------
        X: Tensor to be decomposed
        R: A single rank or best list of ranks to find in obtaining the Tucker SVD
        options: {key:value} map of options to use in the alternating least square optimization
        
        Returns
        -------
        G: Core Tensor
        [F_1, F_2, ...]: Factors of the Unitary Matrices for the modes of the tensor we are querying.
        
        Ref: Kolda and Baer Procedure HOSVD
    """
    
    if isinstance(X, Tensor):
        X = X.data
    
    N = X.ndim
    normX = np.linalg.norm(X)
    
    tol = options.get('tol', 1e-4)
    max_iter = options.get('max_iter', 100)
    dimorder = options.get('dimorder', list(range(N)))
    init     = options.get('init', 'random')
    verbose     = options.get('verbose', True)
    
    if np.isscalar(R):
        R *= np.ones((N, 1), dtype=np.int64)
    U = cell(N)
    
    assert max_iter > 0, "maximum number of iteratons cannot be negative"
    
    if strcmp(init,'random'):
        Uinit = cell(N)
        for n in dimorder[1:]:
            Uinit[n] = np..rand(size(X,n),R[n])
    elif strcmp(init,'nvecs') or strcmp(init,'eigs'):
        # Compute an orthonormal basis for the dominant
        # Rn-dimensional left singular subspace of
        # X_(n) (1 <= n <= N).
        Uinit = cell(N)
        for n in dimorder[1:]:
            info(f'Computing {R[n]} leading e-vectors for factor {n}.')
            Uinit[n] = nvecs(X,n,R[n])
    else:
        raise ValueError('The selected initialization method is not supported.')
        
    U = Uinit
    fit = 0

    if verbose:
        info('Tucker Alternating Least-Squares:')
    
    # Function Motherlode: Iterate until convergence
    for iter in range(max_iter):
        fitold = fit
        
        # iterate over all N modes of the tensor        
        for n in dimorder:
            Utilde = tensor_matrix_mult(X, U, -n, Transpose=True)
            
            'Max the norm of (U_tilde x_n W.T) w.r.t W and keep the'
            'orthonormality of W.'            
            U[n] = nvecs[Utilde, n, R[n]]
        
        # Assemble the approx        
        core = tensor_matrix_mult(Utilde, U, n, Transpose=True)
        
        # Compute the fit
        normresidual = np.sqrt(normX**2 - norm(core)**2)
        fit = 1- (normresidual/normX)
        fitchange = np.abs(fitold-fit)
        
        if iter%5==0:
            info(f"Iter: {iter:2d}, fit: {fit:.4f}, fitdelta: {fitchange:7.1f}")
    
        # Did we converge yet?
        if iter>1 and fitchange < fitchangetol:
            break
        
    T = TuckerTensor(core, U)

    return T

In [371]:
XX = tucker_als(X, [3, 3, 3])

AttributeError: 'list' object has no attribute 'dtype'

In [365]:
__author__ 		= "Lekan Molu"
__copyright__ 	= "2021, Decomposing Level Sets of PDEs"
__credits__  	= "Sylvia Herbert, Ian Abraham"
__license__ 	= "Lekan License"
__maintainer__ 	= "Lekan Molu"
__email__ 		= "patlekno@icloud.com"
__status__ 		= "Fix Tensor Mode Swap in Memory Layout."

import numpy as np.nfrom Utilities import *

class TenMat():
    def __init__(self, T, **options):
        """
            This class provides the boilerpate for matricizing a Tensor.
            
            # TODO: Why does Numpy flip 0-Mode with 1-Mode?
            
            Parameters
            ----------
            T:       A Tensor < see class_tensor.py />.
            options: A bundle class. If it is a dictionary, it is converted to a bundle.
                     It contains the following fields:
                rdims: A numpy/cupy (dtype=intp) index array which specifies the modes of T to 
                       which we map the rows of a matrix, and the remaining 
                       dimensions (in ascending order) map to the columns.
                cdims:  A numpy/cupy (dtype=intp) index array which specifies the modes of T to 
                       which we map the   columns of a matrix, and the 
                       remaining dimensions (in ascending order) map 
                       to the rows.
                cyclic: String which specifies the dimension in rdim which
                        maps to the rows of the matrix, and the remaining 
                        dimensions span the columns in an order specified 
                        by the string argument "cyclic" as follows:

                      'fc' - Forward cyclic.  Order the remaining dimensions in the
                           columns by [rdim+1:T.ndim, 1:rdim-1].  This is the
                           ordering defined by Kiers.

                       'bc' - Backward cyclic.  Order the remaining dimensions in the
                           columns by [rdim-1:-1:1, T.ndim:-1:rdim+1].  
                           This is the ordering defined by De Lathauwer, De Moor, and Vandewalle.

            Calling Signatures
            ------------------
            TenMat(T, options.rdims): Create a matrix representation of a tensor
                T.  The dimensions (or modes) specified in rdims map to the rows
                of the matrix, and the remaining dimensions (in ascending order)
                map to the columns.

            TenMat(T, cdims, Transpose=True): Similar to rdims, but for column
                dimensions are specified, and the remaining dimensions (in
                ascending order) map to the rows.

            TenMat(T, rdims, cdims): Create a matrix representation of
               tensor T.  The dimensions specified in RDIMS map to the rows of
               the matrix, and the dimensions specified in CDIMS map to the
               columns, in the order given.

            TenMat(T, rdim, cyclic): Create the same matrix representation as
               above, except only one dimension in rdim maps to the rows of the
               matrix, and the remaining dimensions span the columns in an order
               specified by the string argument STR as follows:

              'fc' - Forward cyclic.  Order the remaining dimensions in the
                           columns by [rdim+1:T.ndim, 1:rdim-1].  This is the
                           ordering defined by Kiers.

               'bc' - Backward cyclic.  Order the remaining dimensions in the
                           columns by [rdim-1:-1:1, T.ndim:-1:rdim+1].  This is the
                           ordering defined by De Lathauwer, De Moor, and Vandewalle.

            TenMat(T, options=Bundle({rdims, cdims, tsize})): Create a tenmat from a matrix
                   T along with the mappings of the row (rdims) and column indices
                   (cdims) and the size of the original tensor (T.shape).

            Author: Lekan Molux, November 3, 2021
        """
        
        assert isinstance(T, Tensor), 'T must be a tensor class.'
        assert T.data.ndim !=2, "Inp.t Tensor must be 2D."
        assert isinstance(T, Tensor), "T must be of class tensor type."
        
        if not isbundle(options) and isinstance(options, dict):
            options = Bundle(options)
        assert isbundle(options), "options must be of Bundle class."
        
        self.tsize = np.asarray(options.__dict__.get("tsize", T.shape), dtype=np.intp)
        self.rindices = options.__dict__.get("rdims", None)
        self.cindices = options.__dict__.get("cdims", None)
        self.data = T.data        
        self.T= T
        
        tsize = np.asarray(options.__dict__.get("tsize", T.shape), dtype=np.intp)
        rdims = options.__dict__.get("rdims", None)
        cdims = options.__dict__.get("cdims", None)
        data  = T.data
        
        n = numel(tsize)
        
        if np.any(rdims) and np.any(cdims):
            dims_joined = np.concatenate((rdims, cdims))
        elif np.any(rdims) and not np.any(cdims):
            dims_joined = rdims
        elif not np.any(rdims) and np.any(cdims):
            dims_joined = cdims
                        
        if not np.allclose(range(n), np.sort(dims_joined)):
            raise ValueError('Incorrect dimension specifications.')
        elif (np.prod(self.tsize[rdims]) != size(self.data, 0)):
            raise ValueError('T.shape[0] does not match size specified by rdims and shape.')
        elif (np.prod(self.tsize[cdims]) != size(self.data, 1)):
            raise ValueError('T.shape[1] does not match size specified by cdims and shape.')
            
        tsize = T.shape
        n     = T.ndim
        
        tmp = np.zeros((n), dtype=bool)
        tmp.fill(True)
        if np.any(rdims):
            tmp[np.ix_(*rdims)] = False
            cdims = np.nonzero(tmp)
        
        if isfield(options, 'cyclic') and options.cyclic=='T':
            cdims = copy.copy(options.rdims)
            tmp = np.zeros((n), dtype=bool)
            tmp.fill(True)
            tmp[np.ix_(*cdims)] = False            
            rdims = np.nonzero(tmp)
            
        elif isfield(options, 'cyclic') and options.cyclic=='fc':
            rdims = options.rdims
            
            if numel(rdims)!=1:
                raise ValueError('Only one row dimension if third argument is ''fc''.')
            cdims = np.concatenate((np.arange(rdims, n, dtype=np.intp), \
                                    np.arange(rdims-1, dtype=np.intp)), dtype=np.intp)
            
        elif isfield(options, 'cyclic') and options.cyclic=='bc':
            rdims = options.rdims
            
            if numel(rdims)!=1:
                raise ValueError('Only one row dimension if third argument is ''bc''.')
                
            cdims = np.concatenate((np.arange(rdims, -1, 1, dtype=np.intp),\
                                    np.arange(n-1, -1, rdims+1, dtype=np.intp)), dtype=np.intp)
        else:
            raise ValueError('Unrecognized option.')
        
        rdims = options.rdims
        cdims = options.cdims

In [330]:
import numpy as np.nfrom Utilities import *

class TenMat():
    def __init__(self, T, **options):
    #def __init__(self, T, rdims=None, cdims=None, cyclic=None):
        """
        This class provides the boilerpate for matricizing a Tensor.

        Parameters
        ----------
        T:       A Tensor < see class_tensor.py />.
        options: A bundle class. If it is a dictionary, it is converted to a bundle.
                 It contains the following fields:
            rdims: A numpy/cupy (dtype=np.np.intp) index array which specifies the modes of T to 
                   which we map the rows of a matrix, and the remaining 
                   dimensions (in ascending order) map to the columns.
            cdims:  A numpy/cupy (dtype=np.np.intp) index array which specifies the modes of T to 
                   which we map the   columns of a matrix, and the 
                   remaining dimensions (in ascending order) map 
                   to the rows.
            cyclic: String which specifies the dimension in rdim which
                    maps to the rows of the matrix, and the remaining 
                    dimensions span the columns in an order specified 
                    by the string argument "cyclic" as follows:

                  'fc' - Forward cyclic.  Order the remaining dimensions in the
                       columns by [rdim+1:T.ndim, 1:rdim-1].  This is the
                       ordering defined by Kiers.

                   'bc' - Backward cyclic.  Order the remaining dimensions in the
                       columns by [rdim-1:-1:1, T.ndim:-1:rdim+1].  
                       This is the ordering defined by De Lathauwer, De Moor, and Vandewalle.

        Calling Signatures
        ------------------
        TenMat(T, options.rdims): Create a matrix representation of a tensor
            T.  The dimensions (or modes) specified in rdims map to the rows
            of the matrix, and the remaining dimensions (in ascending order)
            map to the columns.

        TenMat(T, cdims, Transpose=True): Similar to rdims, but for column
            dimensions are specified, and the remaining dimensions (in
            ascending order) map to the rows.

        TenMat(T, rdims, cdims): Create a matrix representation of
           tensor T.  The dimensions specified in RDIMS map to the rows of
           the matrix, and the dimensions specified in CDIMS map to the
           columns, in the order given.

        TenMat(T, rdim, cyclic): Create the same matrix representation as
           above, except only one dimension in rdim maps to the rows of the
           matrix, and the remaining dimensions span the columns in an order
           specified by the string argument STR as follows:
           'T' - Transpose.

          'fc' - Forward cyclic.  Order the remaining dimensions in the
                       columns by [rdim+1:T.ndim, 1:rdim-1].  This is the
                       ordering defined by Kiers.

           'bc' - Backward cyclic.  Order the remaining dimensions in the
                       columns by [rdim-1:-1:1, T.ndim:-1:rdim+1].  This is the
                       ordering defined by De Lathauwer, De Moor, and Vandewalle.

        TenMat(T, options=Bundle({rdims, cdims, tsize})): Create a tenmat from a matrix
               T along with the mappings of the row (rdims) and column indices
               (cdims) and the size of the original tensor (T.shape).
               
        Example: 
            X  = np.arange(1, 28).reshape(3,3,3)
            # print('X ', X)
            options = dict(rdims=np.array([2], dtype=np.intp))
            X_1 = TenMat(X, **options)

        Author: Lekan Molux, November 3, 2021
        """

        if not isinstance(T, Tensor):
            T = Tensor(T, T.shape)
            
            
        if not isbundle(options) and isinstance(options, dict):
            options = Bundle(options)
        assert isbundle(options), "options must be of Bundle class."

        self.tsize = np.asarray(options.__dict__.get("tsize", T.shape))
        self.rindices = options.__dict__.get("rdims", None)
        self.cindices = options.__dict__.get("cdims", None)
        self.data = T.data    

        if self.rindices is None and self.cindices is None:
            return

        tsize = np.asarray(options.__dict__.get("tsize", T.shape))
        rdims = options.__dict__.get("rdims", None)
        cdims = options.__dict__.get("cdims", None)
        data  = T.data

        n = T.data.ndim
            
        if len(options)==1:
        #if isfield(options, 'rdims') and not isfield(options, 'cdims'):
            tmp = np.zeros((n), dtype=bool)
            tmp.fill(True)
            tmp[rdims] = False
            cdims = np.nonzero(tmp)[0]
        #elif isfield(options, 'cyclic'):
        elif len(options)>=2: #isfield(options, 'cyclic'):
            if options.cyclic=='T':
                cdims = options.rdims 
                tmp = np.zeros((n,1), dtype=bool)
                tmp.fill(True)
                tmp[cdims] = False
                rdims = np.nonzero(tmp)[0]
            elif options.cyclic=='fc':
                rdims = options.rdims
                if numel(rdims)!=1:
                    raise ValueError(f'Only one row dimension if options.cyclic is ''fc''.')
                cdims = np.concatenate((np.arange(rdims, n, dtype=np.intp), \
                                        np.arange(rdims-1, dtype=np.intp)), dtype=np.intp)
            elif options.cyclic=='bc':
                rdims = options.rdims

                if numel(rdims)!=1:
                    raise ValueError('Only one row dimension if third argument is ''bc''.')

                cdims = np.concatenate((np.arange(rdims-1, dtype=np.intp)[::-1],\
                                        np.arange(rdims, n, dtype=np.intp)[::-1]), dtype=np.intp)
            else:
                raise ValueError('Unrecognized option.')

        else:
            rdims = options.rdims
            cdims = options.cdims

        # Error check
        if not np.array_equal(np.arange(n), np.sort( np.concatenate((rdims, cdims)))):
            raise ValueError('Incorrect specification of dimensions')

        # Permute T so that the dimensions specified by RDIMS come first
        T_Rot = np.transpose(T.data, axes=np.concatenate([rdims, cdims]))
        rprods = np.prod(tsize[rdims])
        np.ods = np.prod(tsize[cdims]) 
        
        self.data     = T_Rot.reshape(rprods, np.ods)
        self.rindices = rdims
        self.cindices = cdims
        self.tsize    = tsize 
        self.T = Tensor(self.data, shape=self.data.shape)
        
    def __call__(self):
        return self.T
        
        

In [356]:
X  = np.arange(1, 28).reshape(3,3,3)
X1 = np.arange(1, 10).reshape(3,3, 1)
X2 = np.arange(10, 19).reshape(3,3, 1)
X3 = np.arange(19, 28).reshape(3,3, 1)
XC = np.array_equal(X, np.concatenate((X1, X2, X3), axis =0))
print('XC: ', XC)
# print('X ', X)
options = dict(rdims=np.array([0], dtype=np.intp))
X_1 = TenMat(X, **options)

XC:  False


In [357]:
X_1.data, X_1.rindices, X_1.cindices, X_1.tsize

(array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24, 25, 26, 27]]),
 array([0]),
 array([1, 2]),
 array([3, 3, 3]))

In [362]:
XX = np.concatenate((X1, X2, X3), 2)
print('XX ', XX)
XX1 = TenMat(XX, rdims=[0])
XX1.data, XX1.rindices, XX1.cindices

XX  [[[ 1 10 19]
  [ 2 11 20]
  [ 3 12 21]]

 [[ 4 13 22]
  [ 5 14 23]
  [ 6 15 24]]

 [[ 7 16 25]
  [ 8 17 26]
  [ 9 18 27]]]


(array([[ 1, 10, 19,  2, 11, 20,  3, 12, 21],
        [ 4, 13, 22,  5, 14, 23,  6, 15, 24],
        [ 7, 16, 25,  8, 17, 26,  9, 18, 27]]),
 [0],
 array([1, 2]))

In [360]:
XX

array([[[ 1],
        [ 2],
        [ 3]],

       [[ 4],
        [ 5],
        [ 6]],

       [[ 7],
        [ 8],
        [ 9]],

       [[10],
        [11],
        [12]],

       [[13],
        [14],
        [15]],

       [[16],
        [17],
        [18]],

       [[19],
        [20],
        [21]],

       [[22],
        [23],
        [24]],

       [[25],
        [26],
        [27]]])